In [31]:
import importlib
from chocodetector import *
chocodetector = importlib.import_module('chocodetector')
importlib.reload(chocodetector)

<module 'chocodetector' from '/home/hthums/master_projects/IAPR/project/project/chocodetector.py'>

In [32]:
ref_dir = 'chocolate-recognition-classic/dataset_project_iapr2025/references/'
train_dir = 'chocolate-recognition-classic/dataset_project_iapr2025/train/'

df_gt = pd.read_csv('chocolate-recognition-classic/dataset_project_iapr2025/train.csv')


df_pred = ChocoDetector(ref_path=ref_dir, train_path=train_dir, features=['gabor', 'lbp'],
                        pca_percentile=0.95, blob_percentile=95, window_size=80, step_size=10, ood_percentile=95, 
                        dt_gt=df_gt)

In [ ]:
y_true = df_gt.iloc[:, 1:].values
y_pred = df_pred.iloc[:, 1:].values

macro_f1, f1_per_class = f1_score_counts(y_true, y_pred)

print(f"Macro F1 Score: {macro_f1:.4f}")

for choc, f1_val in zip(df_gt.columns[1:], f1_per_class):
    print(f"{choc}: {f1_val:.4f}")

Macro F1 Score (count-based): 0.2085
Jelly White: 0.3357
Jelly Milk: 0.2000
Jelly Black: 0.2411
Amandina: 0.4133
Crème brulée: 0.2647
Triangolo: 0.1935
Tentation noir: 0.0000
Comtesse: 0.0333
Noblesse: 0.4478
Noir authentique: 0.2716
Passion au lait: 0.0000
Arabia: 0.1194
Stracciatella: 0.1905


Optimization

In [34]:
from itertools import product

blob_values = [80, 90, 95]
ood_values = [80, 90, 95]
pca_values = [0.95, 0.99, 0.9999]
windows = [60, 80, 100]

results = []
for blob_p, ood_p, pca_p, window in product(blob_values, ood_values, pca_values, windows):
    detector = ChocoDetector(ref_path=ref_dir, train_path=train_dir,
                             features=['gabor', 'lbp'],
                             pca_percentile=pca_p, blob_percentile=blob_p, 
                             ood_percentile=ood_p,
                             window_size=window, step_size=10, dt_gt=df_gt)
    y_pred = detector.iloc[:, 1:].values
    macro_f1, _ = f1_score_counts(y_true, y_pred)
    results.append((macro_f1, blob_p, ood_p, pca_p))

results.sort(reverse=True)
for f1, blob, ood, pca in results:
    print(f"F1={f1:.4f}, blob={blob}, ood={ood}, pca={pca}, window={window}")


KeyboardInterrupt: 